In [1]:
# !pip install ucimlrepo mysql-connector-python sqlalchemy

## Extract ##

In [2]:
# from ucimlrepo import fetch_ucirepo
import pandas as pd

# Fetch dataset
# online_retail = fetch_ucirepo(id=352)
# df = online_retail.data.features
csv_file_path = 'Online Retail.csv'

df = pd.read_csv(csv_file_path)

# Display first few rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Transform ##

In [3]:
# Filter out cancellations
df = df[~df['InvoiceNo'].str.startswith('C')]

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Calculate TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice'] # adding new column

# Drop rows with missing CustomerID
df = df.dropna(subset=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'TotalPrice']) # dropping missing value in this column

# Convert CustomerID to integer
df['CustomerID'] = df['CustomerID'].astype(int)

# Display transformed data
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


## Load ##

In [ ]:
import mysql.connector
from mysql.connector import Error

def batch_insert(df, table_name, batch_size=1000):
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='ecommerce_db',
            user='root',
            password='@Michael2001'
        )
        cursor = connection.cursor()
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        data = [tuple(row) for row in df.itertuples(index=False, name=None)]
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            cursor.executemany(insert_query, batch)
            connection.commit()
            print(f"Inserted {i + len(batch)} rows into {table_name}")
        cursor.close()
        connection.close()
        print("Data loaded successfully.")
    except Error as e:
        print(f"MySQL Error: {e}")

batch_insert(df, 'online_retail')